# SVOD Data Analysis

### ***Metrics Analysis***

---

**PATHs & Libraries**

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

BASE_DIR = Path().resolve()

DATA_PROCESSED_DIR = BASE_DIR / "data" / "processed"

CLEAN_FILE = DATA_PROCESSED_DIR / "clean_data.csv"

df = pd.read_csv(CLEAN_FILE, parse_dates=["fact_date"])

df.head()

,actor_label,country_label,kpi_label_corporate,fact_date,kpi_value
0,AMC Plus,USA,SVOD subscribers,2021-09-30,4258800
1,AMC Plus,USA,SVOD subscribers,2021-12-31,4315500
2,AMC Plus,USA,SVOD subscribers,2022-03-31,4555250
3,AMC Plus,USA,SVOD subscribers,2022-06-30,4938850
4,AMC Plus,USA,SVOD subscribers,2022-09-30,5130650


---

**Grouping data by company for metrics generation**

---

In [2]:
df = df.sort_values(by=["actor_label", "fact_date"]).reset_index(drop=True)

df = df.rename(columns={"kpi_value": "subscribers", "kpi_label_corporate": "KPI", "country_label": "country", "actor_label":"Company"
                })

df.head()

,Company,country,KPI,fact_date,subscribers
0,AMC Plus,USA,SVOD subscribers,2021-09-30,4258800
1,AMC Plus,USA,SVOD subscribers,2021-12-31,4315500
2,AMC Plus,USA,SVOD subscribers,2022-03-31,4555250
3,AMC Plus,USA,SVOD subscribers,2022-06-30,4938850
4,AMC Plus,USA,SVOD subscribers,2022-09-30,5130650


---

**Quarter-over-Quarter Metrics Analysis**

---

1. **Absolute Quarterly Variation (QoQ_abs)**
2. **Percentage Quarterly Variation (QoQ_pct)**
3. **Subscriber Ranking by Date**
4. **Ranking by Absolute and Percentage Growth**
5. **Decline and Stagnation Flags**
6. **Aggregated Metrics per Company (Total Growth: absolute and percentage)**
7. **Count of Positive_Quarters / Negative_Quarters**
8. **Growth Volatility (Standard Deviation of QoQ_pct)**
9. **Compound Annual Growth Rate (CAGR)**


In [3]:
#1 Quarter over Quarter absolute change

df["qoq_abs"] = (
    df.groupby("Company")["subscribers"].diff()
)

df.head(10)

,Company,country,KPI,fact_date,subscribers,qoq_abs
0,AMC Plus,USA,SVOD subscribers,2021-09-30,4258800,NaN
1,AMC Plus,USA,SVOD subscribers,2021-12-31,4315500,56700.0
2,AMC Plus,USA,SVOD subscribers,2022-03-31,4555250,239750.0
3,AMC Plus,USA,SVOD subscribers,2022-06-30,4938850,383600.0
4,AMC Plus,USA,SVOD subscribers,2022-09-30,5130650,191800.0
5,AMC Plus,USA,SVOD subscribers,2022-12-31,5658100,527450.0
6,AT&T TV Now,USA,SVOD subscribers,2021-03-31,625044,NaN
7,AT&T TV Now,USA,SVOD subscribers,2021-06-30,602498,-22546.0
8,Acorn TV,USA,SVOD subscribers,2021-03-31,1745408,NaN
9,Acorn TV,USA,SVOD subscribers,2021-06-30,1867587,122179.0


In [4]:
#2 Quarter over Quarter percentage change

df["qoq_pct"] = (
    df.groupby("Company")["subscribers"]
      .pct_change() * 100
)

df.head(10)

,Company,country,KPI,fact_date,subscribers,qoq_abs,qoq_pct
0,AMC Plus,USA,SVOD subscribers,2021-09-30,4258800,NaN,NaN
1,AMC Plus,USA,SVOD subscribers,2021-12-31,4315500,56700.0,1.331361
2,AMC Plus,USA,SVOD subscribers,2022-03-31,4555250,239750.0,5.555556
3,AMC Plus,USA,SVOD subscribers,2022-06-30,4938850,383600.0,8.421053
4,AMC Plus,USA,SVOD subscribers,2022-09-30,5130650,191800.0,3.883495
5,AMC Plus,USA,SVOD subscribers,2022-12-31,5658100,527450.0,10.280374
6,AT&T TV Now,USA,SVOD subscribers,2021-03-31,625044,NaN,NaN
7,AT&T TV Now,USA,SVOD subscribers,2021-06-30,602498,-22546.0,-3.607106
8,Acorn TV,USA,SVOD subscribers,2021-03-31,1745408,NaN,NaN
9,Acorn TV,USA,SVOD subscribers,2021-06-30,1867587,122179.0,7.000025


In [5]:
#3 Subscriber Ranking by Date

df["rank_subscribers"] = (
    df.groupby("fact_date")["subscribers"]
      .rank(method="dense", ascending=False)
)

df.head(10)

,Company,country,KPI,fact_date,subscribers,qoq_abs,qoq_pct,rank_subscribers
0,AMC Plus,USA,SVOD subscribers,2021-09-30,4258800,NaN,NaN,14.0
1,AMC Plus,USA,SVOD subscribers,2021-12-31,4315500,56700.0,1.331361,16.0
2,AMC Plus,USA,SVOD subscribers,2022-03-31,4555250,239750.0,5.555556,16.0
3,AMC Plus,USA,SVOD subscribers,2022-06-30,4938850,383600.0,8.421053,15.0
4,AMC Plus,USA,SVOD subscribers,2022-09-30,5130650,191800.0,3.883495,15.0
5,AMC Plus,USA,SVOD subscribers,2022-12-31,5658100,527450.0,10.280374,15.0
6,AT&T TV Now,USA,SVOD subscribers,2021-03-31,625044,NaN,NaN,31.0
7,AT&T TV Now,USA,SVOD subscribers,2021-06-30,602498,-22546.0,-3.607106,32.0
8,Acorn TV,USA,SVOD subscribers,2021-03-31,1745408,NaN,NaN,20.0
9,Acorn TV,USA,SVOD subscribers,2021-06-30,1867587,122179.0,7.000025,20.0


In [6]:
#4 Rank by Quarter over Quarter absolute change

df["rank_qoq_abs"] = (
    df.groupby("fact_date")["qoq_abs"]
      .rank(method="dense", ascending=False)
)

df.head(10)

,Company,country,KPI,fact_date,subscribers,qoq_abs,qoq_pct,rank_subscribers,rank_qoq_abs
0,AMC Plus,USA,SVOD subscribers,2021-09-30,4258800,NaN,NaN,14.0,NaN
1,AMC Plus,USA,SVOD subscribers,2021-12-31,4315500,56700.0,1.331361,16.0,24.0
2,AMC Plus,USA,SVOD subscribers,2022-03-31,4555250,239750.0,5.555556,16.0,13.0
3,AMC Plus,USA,SVOD subscribers,2022-06-30,4938850,383600.0,8.421053,15.0,9.0
4,AMC Plus,USA,SVOD subscribers,2022-09-30,5130650,191800.0,3.883495,15.0,15.0
5,AMC Plus,USA,SVOD subscribers,2022-12-31,5658100,527450.0,10.280374,15.0,11.0
6,AT&T TV Now,USA,SVOD subscribers,2021-03-31,625044,NaN,NaN,31.0,NaN
7,AT&T TV Now,USA,SVOD subscribers,2021-06-30,602498,-22546.0,-3.607106,32.0,111.0
8,Acorn TV,USA,SVOD subscribers,2021-03-31,1745408,NaN,NaN,20.0,NaN
9,Acorn TV,USA,SVOD subscribers,2021-06-30,1867587,122179.0,7.000025,20.0,18.0


In [7]:
# 5 Rank by Quarter over Quarter percentage change

df["rank_qoq_pct"] = (
    df.groupby("fact_date")["qoq_pct"]
      .rank(method="dense", ascending=False)
)

df.head(10)

,Company,country,KPI,fact_date,subscribers,qoq_abs,qoq_pct,rank_subscribers,rank_qoq_abs,rank_qoq_pct
0,AMC Plus,USA,SVOD subscribers,2021-09-30,4258800,NaN,NaN,14.0,NaN,NaN
1,AMC Plus,USA,SVOD subscribers,2021-12-31,4315500,56700.0,1.331361,16.0,24.0,94.0
2,AMC Plus,USA,SVOD subscribers,2022-03-31,4555250,239750.0,5.555556,16.0,13.0,47.0
3,AMC Plus,USA,SVOD subscribers,2022-06-30,4938850,383600.0,8.421053,15.0,9.0,25.0
4,AMC Plus,USA,SVOD subscribers,2022-09-30,5130650,191800.0,3.883495,15.0,15.0,57.0
5,AMC Plus,USA,SVOD subscribers,2022-12-31,5658100,527450.0,10.280374,15.0,11.0,19.0
6,AT&T TV Now,USA,SVOD subscribers,2021-03-31,625044,NaN,NaN,31.0,NaN,NaN
7,AT&T TV Now,USA,SVOD subscribers,2021-06-30,602498,-22546.0,-3.607106,32.0,111.0,105.0
8,Acorn TV,USA,SVOD subscribers,2021-03-31,1745408,NaN,NaN,20.0,NaN,NaN
9,Acorn TV,USA,SVOD subscribers,2021-06-30,1867587,122179.0,7.000025,20.0,18.0,45.0


In [8]:
#Flags for Growth, Decline, No Growth

df["is_decline"] = df["qoq_abs"] < 0
df["is_no_growth"] = df["qoq_abs"] == 0

df.head(10)

,Company,country,KPI,fact_date,subscribers,qoq_abs,qoq_pct,rank_subscribers,rank_qoq_abs,rank_qoq_pct,is_decline,is_no_growth
0,AMC Plus,USA,SVOD subscribers,2021-09-30,4258800,NaN,NaN,14.0,NaN,NaN,False,False
1,AMC Plus,USA,SVOD subscribers,2021-12-31,4315500,56700.0,1.331361,16.0,24.0,94.0,False,False
2,AMC Plus,USA,SVOD subscribers,2022-03-31,4555250,239750.0,5.555556,16.0,13.0,47.0,False,False
3,AMC Plus,USA,SVOD subscribers,2022-06-30,4938850,383600.0,8.421053,15.0,9.0,25.0,False,False
4,AMC Plus,USA,SVOD subscribers,2022-09-30,5130650,191800.0,3.883495,15.0,15.0,57.0,False,False
5,AMC Plus,USA,SVOD subscribers,2022-12-31,5658100,527450.0,10.280374,15.0,11.0,19.0,False,False
6,AT&T TV Now,USA,SVOD subscribers,2021-03-31,625044,NaN,NaN,31.0,NaN,NaN,False,False
7,AT&T TV Now,USA,SVOD subscribers,2021-06-30,602498,-22546.0,-3.607106,32.0,111.0,105.0,True,False
8,Acorn TV,USA,SVOD subscribers,2021-03-31,1745408,NaN,NaN,20.0,NaN,NaN,False,False
9,Acorn TV,USA,SVOD subscribers,2021-06-30,1867587,122179.0,7.000025,20.0,18.0,45.0,False,False


In [9]:
# Metrics Aggregated by Company

summary = (
    df.groupby("Company")
      .agg(
        subs_initial=("subscribers", "first"),
        subs_final=("subscribers", "last"),
        max_qoq_abs=("qoq_abs", "max"),
        min_qoq_abs=("qoq_abs", "min"),
        max_qoq_pct=("qoq_pct", "max"),
        min_qoq_pct=("qoq_pct", "min"),
        positive_quarters=("is_decline", lambda x: (~x).sum()),
        negative_quarters=("is_decline", lambda x: x.sum())
      )
      .reset_index()
)

In [10]:
#7 Overall Growth Percentage/Absolute

summary["total_growth_abs"] = summary["subs_final"] - summary["subs_initial"]

summary["total_growth_pct"] = (
    (summary["subs_final"] - summary["subs_initial"])
    / summary["subs_initial"] * 100
)

summary.head(10)

,Company,subs_initial,subs_final,max_qoq_abs,min_qoq_abs,max_qoq_pct,min_qoq_pct,positive_quarters,negative_quarters,total_growth_abs,total_growth_pct
0,AMC Plus,4258800,5658100,527450.0,56700.0,10.280374,1.331361,6,0,1399300,32.856673
1,AT&T TV Now,625044,602498,-22546.0,-22546.0,-3.607106,-3.607106,1,1,-22546,-3.607106
2,Acorn TV,1745408,2655000,247500.0,26682.0,10.280374,1.335223,8,0,909592,52.113431
3,Allblk,69583,119180,16361.0,2435.0,21.949583,3.499418,8,0,49597,71.277467
4,Amazon Prime Video,90037500,98820754,4455955.0,-1818757.0,4.999999,-1.999999,7,1,8783254,9.755106
5,Apple TV+,18909651,28321810,3191003.0,-1223218.0,14.999997,-5.000000,6,2,9412159,49.774367
6,Arrow Player,20226,33568,2336.0,1518.0,7.529090,7.479508,8,0,13342,65.964600
7,AsianCrush,27309,31370,615.0,547.0,2.003003,1.999085,8,0,4061,14.870555
8,BET+,882331,2338821,304047.0,131843.0,14.942578,14.942488,8,0,1456490,165.072971
9,Bally Sports+,55000,82500,27500.0,27500.0,50.000000,50.000000,2,0,27500,50.000000


In [11]:
#9 Growth Volatility

volatility = (
    df.groupby("Company")["qoq_pct"]
      .std()
      .reset_index()
      .rename(columns={"qoq_pct": "volatility_pct"})
)

summary = summary.merge(volatility, on="Company", how="left")


In [12]:
#10 CAGR

df_dates = df.groupby("Company")["fact_date"].agg(["min", "max"]).reset_index()

df_dates["years"] = (
    (df_dates["max"] - df_dates["min"]).dt.days / 365
)

summary = summary.merge(df_dates[["Company", "years"]], on="Company")

summary["cagr"] = (
    (summary["subs_final"] / summary["subs_initial"]) ** (1 / summary["years"]) - 1
)

df_dates.head(10)

,Company,min,max,years
0,AMC Plus,2021-09-30,2022-12-31,1.252055
1,AT&T TV Now,2021-03-31,2021-06-30,0.249315
2,Acorn TV,2021-03-31,2022-12-31,1.753425
3,Allblk,2021-03-31,2022-12-31,1.753425
4,Amazon Prime Video,2021-03-31,2022-12-31,1.753425
5,Apple TV+,2021-03-31,2022-12-31,1.753425
6,Arrow Player,2021-03-31,2022-12-31,1.753425
7,AsianCrush,2021-03-31,2022-12-31,1.753425
8,BET+,2021-03-31,2022-12-31,1.753425
9,Bally Sports+,2022-09-30,2022-12-31,0.252055


---

**Export Final Datasets**

---

In [13]:
# Quarterly Metrics Export

timeseries_file = DATA_PROCESSED_DIR / "timeseries_metrics.csv"
df.to_csv(timeseries_file, index=False)
timeseries_file

# Company Summary Metrics Export

summary_file = DATA_PROCESSED_DIR / "company_summary.csv"
summary.to_csv(summary_file, index=False)
summary_file

WindowsPath('D:/GitHub/DataProjects/Dataxis/data/processed/company_summary.csv')